# AWS DynamoDB 

## Principales características

- Servicio completamente administrado (serverless), no hay ningún tipo de capacidad que aprovisionar. No hay servidores que administrar.
- Se paga por la cantidad de almacenamiento que se esté consumiendo y por las operaciones de lectura y escritura. Tiene una capa gratuita de 25 gigabytes de almacenamiento de por vida (no solamente con el primer año de capa gratuita).
- El rendimiento va a escalar en base a lo que se le indique, es decir, sus modos de rendimiento en lectura y modo de escritura, pueden ser bajo demanda o bajo un rendimiento aprovisionado.
- Consultas de muy baja latencia (de milisegundos de un solo dígito).
- Datos cifrados en reposo por defecto utilizando claves de KMS.
- DynamoDB Streams captura una secuencia en orden cronológico de las modificaciones de los elementos en una tabla de DynamoDB y almacena esta información en un log durante un máximo de 24 horas. Esta cualidad permite propagar todos los cambios y operaciones que se hagan de modificación de los elementos de la tabla en tiempo real.
- Tiene respaldos automatizados.
- Point-in-time recovery (PITR): permite recuperar la tabla de DynamoDB, en un momento en específico dentro de los último 35 días.
- Desde el punto de vista de la seguridad, se integra con IAM, lo que permite determinar mediante políticas basadas en la identidad (y desde hace poco, políticas basadas en el recurso) asignadas a tablas de DynamoDB, cuáles operaciones se pueden o no realizar. Estas operaciones se pueden restringir a nivel de (columnas) atributos en una misma tabla. Se puede granular los niveles de acceso de forma detallada.
- Las tablas de DynamoDB para alta disponibilidad, están replicadas en 3 ubicaciones diferentes (existen 3 copias de las tablas de DynamoDB).
- No es un esquema fijo, es un esquema semiestructurado.
- Soporta transacciones, es decir, ejecución de instrucciones como todo o nada. En una modalidad trasaccional, al realizar una lectura se consume el doble de RCU (read capacity unit) que una lectura fuertemente consistente.
- Se puede definir autoescalado de las RCU (read capacity unit) y WCU (write capacity unit).

## Conceptos básicos en DynamoDB

### Tablas

Tablas: (no existe el concepto de base de datos, sino de tablas) ellas no se relacionan, no existe dentro del sistema del gestor de base de datos (SGBD). Si se requiere implementar algún tipo de restricción de integridad, se tiene que hacer mediante código, porque no existe dentro del modelo gestor de la base de datos. En las tablas hay que definir un campo o un atributo que se denomina clave de partición (función hash), es requerido, toda tabla debe tener una clave de partición.

#### Clave de partición (atributo o clave hash):
La clave de partición es un atributo específico de cada registro en una tabla.
Es obligatoria al momento de crear toda tabla.
Es un valor único que identifica un registro específico en una tabla. Es como un número de identificación único (ID) que se asigna a cada elemento (registro) en la tabla.
Esta clave determina físicamente cual es partición donde se encuentra el elemento, mediante una función hash que implementa internamente DynamoDB, para ubicar el elemento en una partición u en otra.
No es como la clave primaria porque no es única.
Una vez que se crea, por defecto ya no se debe definir ningún otro atributo. El resto de atributos se van añadiendo, conforme se van introduciendo elementos (filas) dentro de la tabla (esquema semi-estructurado).
Permite distribuir los datos de manera eficiente, mejorar el rendimiento y facilitar el acceso a la información. Elegir una buena clave de partición, asegura un buen desempeño y escalabilidad para la aplicación.
El término atributo hash alude al uso de una función hash interna en DynamoDB para distribuir los elementos de datos de manera uniforme entre las particiones, según sus valores de clave de partición. Esta distribución uniforme pretende que los datos estén repartidos uniformemente entre todas las particiones, y no que se concentren en una partición en específico (partición caliente o hot partition). Una partición no debe escalar más allá de la capacidad de lectura o escritura totales partido por el número de particiones.
DynamoDB utiliza esta clave para distribuir los datos entre diferentes nodos o servidores. Esto ayuda a equilibrar la carga de trabajo y a garantizar que el acceso a los datos sea rápido y eficiente.
Cuando se define en la tabla la clave de partición, cada valor de esta clave irá en una partición diferente.

### Particiones
- Determina la ubicación de los elementos. Las particiones son muy importantes porque definen el modo de búsqueda de un elemento, el cual puede ser:
- En una partición concreta. La forma más eficiente de consultar una tabla.
Todas las particiones (modo ineficiente).

## Crecimiento y Particiones
- **Crecimiento de las particiones**: Las particiones de una tabla en DynamoDB crecen conforme se añaden elementos.  
- **Tamaño máximo de una partición**: 10 GB.  
  - Cuando se alcanza este límite, DynamoDB automáticamente realiza un **split** (divide la partición) y ajusta la función **hash**.

## Alta Disponibilidad
- Las tablas de DynamoDB están replicadas en **tres ubicaciones diferentes** para garantizar su alta disponibilidad. Esto significa que hay **tres copias de las tablas**.

## Clave de Ordenación o Clave Range (Opcional)
- Determina cómo se ordenan los elementos dentro de una partición que tienen el mismo valor **hash** para la clave de partición.  
- Los elementos se ordenan según esta clave, y su uso es opcional.

## Elementos o Registros (Filas)
- Son las **filas** de una tabla y contienen los datos reales de cada conjunto de atributos.  
- Se expresan en formato **JSON**, con un tamaño máximo de **400 KB** por documento.  
  - Este límite es una **cuota fija** que no puede aumentarse.  
  - Para información de mayor tamaño, se recomienda almacenar los datos en **Amazon S3** y colocar un enlace al objeto en S3.

## Atributos
- Son las **columnas** de una tabla que definen qué tipo de datos se almacenan en cada elemento.

## Clave Principal o Clave Primaria
Cada elemento en la tabla debe tener una clave primaria única. Existen dos tipos:  
1. **Clave Principal Simple (Hash Key o Clave de Partición)**:  
   - Formada únicamente por el campo de la clave de partición.  
2. **Clave Principal Compuesta (Clave de Partición y Clave de Ordenación)**:  
   - La unicidad la proporciona la combinación de ambos valores.

## Introducción de Elementos en una Tabla
- En las interfaces de bajo nivel, cada atributo del elemento debe tener definido su tipo de dato, por ejemplo:  
  - **Entero**, **Número (N)**, **String (S)**, **Booleano**.  
  - Tipos estructurados como **Mapas (documentos JSON)** o **listas de elementos** también deben especificar el tipo de dato.  


## El tipo de dato de un atributo determinado, de un elemento en una tabla de DynamoDB puede ser a su vez, otro documento JSON.

### Buscar elementos en DynamoDB
* **Buscar el elemento en todas las particiones.** Modo ineficiente.
* **Buscar el elemento en una partición concreta.** Forma más eficiente con consulta una tabla de DynamoDB.

Los elementos se pueden consultar o buscar en base a una partición.

| Clave Hash o clave de partición (partition key) | Sort Key (clave de ordenación o clave Range) |
|---|---|
| UserID | El atributo que es la clave de ordenación de índice |


### Capacidad de Almacenamiento de DynamoDB:
* Permite escalar hasta petabytes de información, de forma completamente administrada, sin aprovisionamiento de antemano.

### Capacidad de Lectura y Escritura en DynamoDB:
Mide la capacidad de lectura y escritura mediante 2 parámetros:
1. **RCU (read capacity unit):** capacidad de leer un bloque de hasta 4kilobyte (KB) por segundo.
    * 1 RCU lee 1 bloque de hasta 4 KB por segundo.
    * 1 RCU lee 1 bloques de 4 KB
2. **WCU (write capacity unit):** capacidad de escribir un bloque de hasta 1 KB por segundo.

**Ejemplo:** Se tienen elementos de tamaño 13 KB, ¿qué capacidad de lectura fuertemente consistente se necesitaría aprovisionar para leer 7 elementos por segundo o por unidad de tiempo?
* **Se requiere para un modelo de lectura fuertemente consistente**
* Para calcular esto: (se calcula de esta forma, es decir, fuertemente consistente, si por ejemplo se ha escrito un elemento e inmediatamente después se está realizando una lectura y se requiere que la misma devuelva el elemento actualizado)
    * Las RCU leen en bloques de 4 KB.
    * Primero hay que definir el número de bloques que se necesita para aprovisionar el tamaño de los elementos (13 KB). Y luego se multiplica este número de bloques, por la cantidad de elementos que se necesitan leer (7 elementos).


Se necesita leer 7 elementos por segundo, de cada uno de 13 KB. 4 RCU bloques x 7 elementos por segundo = 28 RCU para garantizar la lectura, en un modelo de lectura fuertemente consistente.

Se necesita que la tabla tenga 28 RCU para garantizar una lectura: 

| 1 bloque de 4 KB (1 RCU) | 1 bloque de 4 KB (1 RCU) | 1 bloque de 4 KB (1 RCU) | 1 bloque de 4 KB (1 RCU) | = 4 RCU / bloques |
|---|---|---|---|---|
|4 KB de elemento| 8 KB de elemento | 12 KB de elemento | 16 KB de elemento | 

Con este mismo ejemplo, pero dentro de un modelo de lectura de consistencia eventual (lecturas eventualmente consistentes): con 14 RCU sería suficiente.

**Ejemplo:** Se tienen elementos de tamaño 13 KB, ¿qué capacidad de lectura de consistencia eventual se necesitaría aprovisionar para leer 7 elementos por segundo o por unidad de tiempo?

| 1 bloque de 4 KB (1/2 RCU) | 1 bloque de 4 KB (1/2 RCU) | 1 bloque de 4 KB (1/2 RCU) | 1 bloque de 4 KB (1/2 RCU) | = 2 RCU |
|---|---|---|---|---|
| 4 KB de elemento | 8 KB de elemento | 12 KB de elemento | 16 KB de elemento |  |

14 RCU.

Por defecto, las lecturas a DynamoDB son eventualmente consistentes (igualmente bloques de 4K y se necesita 1/2 RCU). Las lecturas fuertemente consistente se deben indicar previamente.

* **Modelo de lectura fuertemente consistente:** Cuando se requiere que la lectura devuelva a el elemento actualizado. Siempre encontraras la versión más actualizada de los elementos. Siempre obtienes los datos más recientes, no hay retrasos.
* **Modelo de lectura de consistencia eventual (lecturas eventualmente consistentes):** Cuando la aplicación no necesita lectura fuertemente consistente y se puede permitir cierta desactualización de los datos, se puede aplicar un modelo de consistencia eventual. En este caso, cuando se realiza una lectura, no siempre se obtendrá el dato más reciente al instante. Puede haber un pequeño retraso mientras los cambios se propagan a través del sistema. Estas lecturas utilizan aproximadamente la mitad de la RCU. Con lecturas eventualmente consistentes, las lecturas son a la mitad de lo provisionado en una capacidad de lectura.

2.- **WCU (write capacity unit):** Unidad de capacidad de escritura: cantidad de escritura de bloques de 1KB x unidad de tiempo.

Ejemplo: Se requiere escribir 7 elementos x segundo y cada elemento es de 13 KB,
¿cuántas WCU se necesitan?

Respuesta: 13Kb x 7 (elementos x seg.) = 91 WCU

 Índices: Al definirse una clave primaria en una tabla de DynamoDB, los
elementos están particionados en función de esa clave de partición. Y al
definirse la clave de ordenación, los elementos están ordenados dentro de cada
partición.

Existen 2 tipos de índices en DynamoDB: (el índice principal está definido sobre la
clave de partición).

Índices secundarios locales (LSI - Local Secondary Indexes): tiene que tener la
misma clave de partición de la tabla a partir de la cual se crea. No se puede cambiar
el particionado de la tabla, es decir, la clave Hash o clave de partición (partition key),
sino la clave reordenada de partición (criterio de ordenación), es decir, la sort key
(clave de ordenación o clave Range).

Solamente se pueden crear en el momento de la creación de la tabla. No se pueden
crear luego que la tabla haya sido creada.

El acceso a una tabla a través de un LSI, consume de la misma capacidad de lectura
(RCU) y escritura (WCU) que la tabla principal. Se comparten las RCU y WCU.

Índices secundarios globales (GSI - Global Secondary Indexes): se puede crear a
posterior, una vez creada la tabla. Se puede redefinir tanto la la clave Hash o clave
de partición (partition key), y la la sort key (clave de ordenación o clave Range).

De igual forma, hay que definir una capacidad de escritura y lectura independiente a
la de la tabla, no comparte RCU ni WCU de la tabla, sino que tiene su propia
capacidad de RCU y WCU aprovisionada.

Modo de capacidad aprovisionada en DynamoDB (aprovisionamiento de capacidad)

Ejemplo: Se aprovisiona para una tabla de DynamoDB una capacidad de 1000 RCU y
500 WCU, capacidad que se reparte de forma uniforme entre todas las particiones,
lo que significa, que si la tabla de DynamoDB tiene 10 particiones, cada partición
tendrá 100 RCU y 50 WCU.

Otro ejemplo, es que se aprovisiona una tabla con una capacidad de 400 WCU en 4
particiones. Estas 400 WCU se redistribuyen de forma uniforme en las 4 particiones,
correspondiendo de esta manera a 100 WCU a cada una de las particiones.

Capacidad de adaptación y Capacidad de ampliación (Burst Capacity and Adaptive
Scaling)

Para minimizar las limitaciones debidas a las excepciones de rendimiento,
DynamoDB utiliza la capacidad de ampliación para gestionar los picos de uso.
DynamoDB utiliza la capacidad de adaptación para adaptarse a patrones de acceso
irregulares.

Capacidad de adaptación (adaptive scaling): esta cualidad solucionó el antiguo
fenómeno de Hot Partition, que se provocaba, por ejemplo, por un pico de carga de
trabajo o por un problema en el diseño de la tabla (que la clave de partición esté mal
diseñada, lo que hace que la mayor parte de las consultas, lecturas o escrituras estén
impactando sobre una partición en específico).

La capacidad adaptativa (traspasar capacidad de lectura y escritura RCU / WCU no
utilizada por el resto de particiones puntualmente, a una partición que lo requiera).
Permite que temporalmente, una partición pueda exceder su capacidad de escritura
y lectura (RCU / WCU) aprovisionada (siempre y cuando el resto de las particiones no
estén consumiendo esa capacidad). Esta cualidad está pensada para picos de cargas
de trabajo puntuales, no para solucionar un problema estructural, como por ejemplo,
una deficiente definición de una clave de partición.

Capacidad de ampliación (burst capacity): permite acumular la capacidad de lectura
o escritura, y utilizarla en un escalado en un momento determinado. Esta capacidad
solamente se guarda durante 5 minutos. Se acumula la capacidad que no se haya
utilizado en los últimos 5 minutos, con el fin de disponer de esta en un momento
determinado que se requiera.

Amazon Web Service se reserva el derecho de consumir esa capacidad de lectura o
escritura acumulada (capacidad de ampliación) en operaciones de background o de
mantenimiento de la tabla.

---

**Fuentes:**  
- AWS Documentación.  
- AWS Skill Builder.